In [ ]:
%matplotlib ipympl

import ipywidgets as widgets
from IPython.display import display

from experimental.beacon_sim.experiment_results_pb2 import ExperimentResult
from experimental.beacon_sim.experiment_config_pb2 import ExperimentConfig
from experimental.beacon_sim.world_map_config_pb2 import WorldMapConfig
from planning.road_map_pb2 import RoadMap
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib as mpl
from pathlib import Path

In [ ]:
file_name = '/tmp/exp_1.pb'
with open(file_name, 'rb') as file_in:
    data = ExperimentResult()
    data.ParseFromString(file_in.read())

In [ ]:
def protobuf_to_df(pb):
    out = defaultdict(list)
    i = 0
    for result in pb.results:
        out['elapsed_time'].append(result.elapsed_time.count / 1e9)
        out['trial_id'].append(result.trial_id)
        out['planner_id'].append(result.planner_id)
        if result.HasField('plan'):
            out['plan'].append(list(result.plan.nodes))
            out['log_prob_mass'].append(result.plan.log_prob_mass)
            out['expected_det'].append(result.plan.expected_det)
        else:
            out['plan'].append(None)
            out['log_prob_mass'].append(None)
            out['expected_det'].append(None)
    return pd.DataFrame(out)
        

In [ ]:
df = protobuf_to_df(data)
planners = data.planner_names

In [ ]:
data.planner_names

In [ ]:
df

In [ ]:


axes = df.plot.hist(by='planner_id', column='expected_det', bins=50, figsize=(13,10))
for i, ax in enumerate(axes):
    ax.set_yscale('log')
    ax.set_title(planners[i])
axes[-1].set_xlabel('Expected Determinant')
plt.suptitle('Expected Determinant')
plt.tight_layout()



axes = df.plot.hist(by='planner_id', column='elapsed_time', bins=50, figsize=(13,10))
for i, ax in enumerate(axes):
    ax.set_yscale('log')
    ax.set_title(planners[i])
axes[-1].set_xlabel('Elapsed Time (s)')
plt.suptitle('Elasped Time')
plt.tight_layout()

axes = df.plot.hist(by='planner_id', column='log_prob_mass', bins=50, figsize=(13,10))
for i, ax in enumerate(axes):
    ax.set_yscale('log')
    ax.set_title(planners[i])
axes[-1].set_xlabel('Log Prob Mass')
plt.suptitle('Log Prob Mass')
plt.tight_layout()

In [ ]:
import numpy as np
np.exp(-1)

In [ ]:

def load_proto(path: Path, proto_class):
    with open(path, 'rb') as file_in:
        out = proto_class()
        out.ParseFromString(file_in.read())
    return out

def load_config(experiment_config: ExperimentConfig, base_path: Path):
    # Load world map
    world_map = load_proto(base_path / experiment_config.map_config_path, WorldMapConfig)
    # Load road map
    road_map = load_proto(base_path / experiment_config.road_map_path, RoadMap)
    return world_map, road_map

def plot_beacon_locations(world_map_proto):
    beacons = []
    for beacon in  world_map_proto.correlated_beacons.beacons:
        beacons.append([beacon.pos_x_m, beacon.pos_y_m])
    beacons = np.array(beacons)

    plt.plot(beacons[:, 0], beacons[:, 1], 'rs')

def plot_road_map(road_map_proto):
    nodes = []
    for node in road_map_proto.points:
        nodes.append(node.data)
    adj = np.array(road_map_proto.adj.data).reshape(
        road_map_proto.adj.num_rows, road_map_proto.adj.num_cols)


    segs = []
    for i in range(adj.shape[0]):
        for j in range(i+1, adj.shape[1]):
            if adj[i, j]:
                segs.append([nodes[i], nodes[j]])

    
    nodes = np.array(nodes)
    segs = np.array(segs)
    road_map_color = (0.8, 0.8, 0.8, 1.0)
    line_collection = mpl.collections.LineCollection(segs, colors=road_map_color, linewidths=2.0)
    plt.plot(nodes[:, 0], nodes[:, 1], 's', color=road_map_color, markersize=5)
    plt.gca().add_artist(line_collection)

def make_trial_plotter(data: ExperimentResult, road_map: RoadMap):
    df = protobuf_to_df(data)
    planner_names = data.planner_names
    start_goals = data.start_goal

    drawn_artists = []

    def plotter(*, trial_idx, planner_idx):
        nonlocal drawn_artists
        for artist in drawn_artists:
            artist.remove()
        drawn_artists = []

        # Draw start and goal
        drawn_artists += plt.plot(data.start_goal[trial_idx].start.data[0],
                       data.start_goal[trial_idx].start.data[1], 'gs')
        drawn_artists += plt.plot(data.start_goal[trial_idx].goal.data[0],
                       data.start_goal[trial_idx].goal.data[1], '*', color='tab:cyan', markersize=10)

        # Draw plan
        trial_idx_mask = df['trial_id'] == trial_idx
        planner_idx_mask = df['planner_id'] == planner_idx
        mask = np.logical_and(trial_idx_mask, planner_idx_mask)
        plan = df[mask].iloc[0].loc['plan']
        segs = []
        for (start, end) in zip(plan[:-1], plan[1:]):
            seg = []
            for id in [start, end]:
                if id == -1:
                    seg.append(data.start_goal[trial_idx].start.data)
                elif id == -2:
                    seg.append(data.start_goal[trial_idx].goal.data)
                else:
                    seg.append(road_map.points[id].data)
            segs.append(seg)       
        segs = np.array(segs)    
        plan_color = (0.718, 0.0, 1.0, 1.0)
        line_collection = mpl.collections.LineCollection(segs, colors=plan_color)
        plt.gca().add_artist(line_collection)
        drawn_artists.append(line_collection)
            
    plotter(trial_idx=0, planner_idx=0)
    return plotter

def create_trial_widgets(data, trial_plotter):
    trial_slider = widgets.IntSlider(min=0, max=len(data.start_goal)-1, step=1, value=0, description='Trial Idx:')
    planner_dropdown = widgets.Dropdown(options={
        planner_name: planner_id for planner_id, planner_name in enumerate(data.planner_names)},
        description='Planner Name'
    )

    def on_change(change):
        trial_plotter(trial_idx=trial_slider.value, planner_idx=planner_dropdown.value)
    
    trial_slider.observe(on_change, names='value')
    planner_dropdown.observe(on_change, names='value')
    return widgets.Box([trial_slider, planner_dropdown])

def visualize_trials(data: ExperimentResult, base_path: Path):
    experiment_config = data.experiment_config
    world_map, road_map = load_config(experiment_config, base_path)

    plt.figure()
    
    # Plot the beacon locations
    plot_beacon_locations(world_map)

    # Plot the RoadMap
    plot_road_map(road_map)

    # Create gui widgets for selecting trial
    trial_plotter = make_trial_plotter(data, road_map)
    display(create_trial_widgets(data, trial_plotter))

    # Create gui widget for selecting beacon configuration

In [ ]:
visualize_trials(data, Path('~/code/robot/experimental/beacon_sim/experiment_configs').expanduser())